In [1]:
# from functools import partial
from pathlib import Path
# from typing import Optional, Tuple
# import cv2
# import fire
import numpy as np
import torch
import torch.nn.functional as F
from accelerate import Accelerator
from PIL import Image
from scipy.sparse.linalg import eigsh
# from sklearn.cluster import KMeans, MiniBatchKMeans
# from sklearn.decomposition import PCA
# from torchvision.utils import draw_bounding_boxes
from tqdm import tqdm
import extract_utils as utils
from torch.utils.data import Dataset, DataLoader
from lightly.loss import NegativeCosineSimilarity
from lightly.models.modules import SimSiamPredictionHead, SimSiamProjectionHead
from torch import nn
import torchvision
import datetime, dateutil
from sklearn.decomposition import PCA
from scipy.ndimage import affine_transform

## Extract Eigen Vectors

In [2]:
images_root="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/images"
features_dir="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/features/dino_vits16"
output_dir="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/eigs_dot1PCANetpredconv1Dsimsiam_ds_10_model_jn"
which_matrix= 'laplacian'
which_color_matrix= 'knn'
which_features= 'k'
normalize=True
threshold_at_zero=True
lapnorm= True
K= 5
image_downsample_factor = None
image_color_lambda = 0.0
multiprocessing = 0
batch_size=2
epochs=10

## ResNet Residual Block

In [3]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels=1, out_channels=1, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=in_channels, out_channels=64, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm1d(64)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(64, out_channels * self.expansion, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels * self.expansion)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)
        # print("Before squeezing, out shape=", out.shape)
        out =  out.squeeze().to('cuda')
        # print("After squeezing, out shape=", out.shape)

        return out

## Incorporating SimSiam

In [4]:
# Define a custom dataset class
class Feature_Dataset(Dataset):
    def __init__(self, features):
        self.features = features

    def __getitem__(self, index):
        return self.features[index]

    def __len__(self):
        return len(self.features)

In [5]:
class SimSiam(nn.Module):
    def __init__(self, train=True):
        super().__init__()
        # self.projection_head = SimSiamProjectionHead(feats.shape[1], 128,feats.shape[1])
        self.train=train
        self.projection_head = BasicBlock()
        # self.projection_head = ResidualBlock(feats.shape[1], feats.shape[1])
        # self.prediction_head = SimSiamPredictionHead(feats.shape[1], 128, feats.shape[1])
        self.prediction_head = BasicBlock()

    def forward(self, x,train=True):
        z = self.projection_head(x)
        # print("z.shape", z.shape)
        z_new=z.view(feats.shape[0],1 , 128)
        # if train:
        #     z_new = z.view(2, 1, 384)
        # else:
        #     z_new=z.view(feats.shape[0],1 , 384)
            # print(z_new.shape)
        p = self.prediction_head(z_new)
        z_new.detach()
        # if train:
        #     z_new = z.view(2, 1, 384)
        #     # print(z_new.shape)
        #     p = self.prediction_head(z_new)
        #     z_new.detach()
        # else:
        #     p = self.prediction_head(z)
        # z = z.detach()
        return z_new, p

In [6]:
# utils.make_output_dir(output_dir)
# inputs = list(enumerate(sorted(Path(features_dir).iterdir())))
# model_simsiam = SimSiam()
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model_simsiam.to(device)
# criterion = NegativeCosineSimilarity()
# optimizer = torch.optim.SGD(model_simsiam.parameters(), lr=0.06)
# for inp in tqdm(inputs):
#     index, features_file = inp
#     print(index, features_file)
#      # Load
#     data_dict = torch.load(features_file, map_location='cpu')
#     # print(data_dict.keys())   #['k', 'indices', 'file', 'id', 'model_name', 'patch_size', 'shape']
#     # print("shape=", data_dict['shape'], "k shape", data_dict['k'].shape, "patch_size=", data_dict['patch_size'])
#     image_id = data_dict['file'][:-4]
#     print(image_id)
#     # Load
#     output_file = str(Path(output_dir) / f'{image_id}.pth')
#     if Path(output_file).is_file():
#         print(f'Skipping existing file {str(output_file)}')
#         # break
#         # return  # skip because already generated
#
#     # Load affinity matrix
#     feats = data_dict[which_features].squeeze().cuda()
#     # print("Without normalizing, Features Shape is",feats.shape)
#     if normalize:
#         feats = F.normalize(feats, p=2, dim=-1)
#
#      # Get sizes
#         B, C, H, W, P, H_patch, W_patch, H_pad, W_pad = utils.get_image_sizes(data_dict)
#         if image_downsample_factor is None:
#             image_downsample_factor = P
#         H_pad_lr, W_pad_lr = H_pad // image_downsample_factor, W_pad // image_downsample_factor
#
#         # Upscale features to match the resolution
#         if (H_patch, W_patch) != (H_pad_lr, W_pad_lr):
#             feats = F.interpolate(
#                 feats.T.reshape(1, -1, H_patch, W_patch),
#                 size=(H_pad_lr, W_pad_lr), mode='bilinear', align_corners=False
#             ).reshape(-1, H_pad_lr * W_pad_lr).T
#
#         ### Feature affinities
#         # print("Without normalizing, Features Shape is",feats.shape)
#
#         W_feat_ds = (feats @ feats.T)
#         # feat_list.append(feats)
#         # feat_dataset = Feature_Dataset(feats)
#         # if feats.shape[0]%2==0:
#         #     features_dataloader = DataLoader(feat_dataset, batch_size=batch_size, shuffle=True)
#         # else:
#         #     features_dataloader = DataLoader(feat_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
#         print("Starting Training")
#         x0=feats
#         x0 = x0.unsqueeze(0).to(device)
#         # x0 = x0.unsqueeze(1).to(device)
#         print("After Unsqueezing x0 shape=", x0.shape)
#         # print("x0 shape=", x0.shape)
#         x1=torchvision.transforms.RandomAffine(0)(x0)
#         print("After Unsqueezing x1 shape=", x1.shape)
#          # x0 = x0.squeeze(0).to(device)
#         # print("batch_size=",batch_size)
#         # x0_new = torch.tensor(x0).view(batch_size, 1, 1, 384)
#         # x0_new = torch.tensor(x0).view(batch_size, 1, 384)
#         x0_new = x0.view(feats.shape[0], 1, 384)
#         # print("After viewing x0 shape=", x0_new.shape)
#         # print("x0.shape=", x0.shape)
#         # x1 = x1.squeeze(0).to(device)
#         # x1 = torch.tensor(x1).view(batch_size, 1, 1,384)
#         x1_new = x1.view(feats.shape[0], 1, 384)
#         # print("After viewing x1 shape=", x1_new.shape)
#         for epoch in range(epochs):
#             total_loss = 0
#             z0, p0 = model_simsiam(x0_new, True)
#             # print("zo.shape", z0.shape)
#             # print("p0.shape", p0.shape)
#             z1, p1 = model_simsiam(x1_new, True)
#             loss = 0.5 * (criterion(z0, p1) + criterion(z1, p0))
#             total_loss += loss.detach()
#             loss.backward()
#             optimizer.step()
#             optimizer.zero_grad()
#         avg_loss = total_loss / feats.shape[0]
#         print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")
#


## Model Based Optimization

In [15]:

now = datetime.datetime.now(dateutil.tz.tzlocal())
timestamp = now.strftime('%Y_%m_%d_%H_%M_%S')
model_path="/home/phdcs2/Hard_Disk/Datasets/Deep-Spectral-Segmentation/data/object-segmentation/ECSSD/weights/dot1_PCANetpredconv1d_ds_10_model_jn%s.pt" % (timestamp)
utils.make_output_dir(output_dir)
inputs = list(enumerate(sorted(Path(features_dir).iterdir())))
model_simsiam = SimSiam()
device = "cuda" if torch.cuda.is_available() else "cpu"
model_simsiam.to(device)
criterion = NegativeCosineSimilarity()
optimizer = torch.optim.SGD(model_simsiam.parameters(), lr=0.06)
pca = PCA(n_components=128)

print("Starting Training")
best_loss = float('inf')
for epoch in range(epochs):
    print("epoch", epoch+1)
    total_loss = 0
    for inp in tqdm(inputs):
        index, features_file = inp
         # Load
        data_dict = torch.load(features_file, map_location='cpu')
        # image_id = data_dict['file'][:-4]
        # Load
        feats = data_dict[which_features].squeeze().cuda()
        # print("Without normalizing, Features Shape is",feats.shape)
        if normalize:
            feats = F.normalize(feats, p=2, dim=-1)

        # Get sizes
        B, C, H, W, P, H_patch, W_patch, H_pad, W_pad = utils.get_image_sizes(data_dict)
        if image_downsample_factor is None:
            image_downsample_factor = P
        H_pad_lr, W_pad_lr = H_pad // image_downsample_factor, W_pad // image_downsample_factor

        # Upscale features to match the resolution
        if (H_patch, W_patch) != (H_pad_lr, W_pad_lr):
            feats = F.interpolate(
                feats.T.reshape(1, -1, H_patch, W_patch),
                size=(H_pad_lr, W_pad_lr), mode='bilinear', align_corners=False
            ).reshape(-1, H_pad_lr * W_pad_lr).T

        ### Model-Based Optimization
        x0=feats
        x0_arr=x0.cpu()
        x0_arr= pca.fit_transform(x0_arr)


        # Define the affine transformation parameters
        scale = np.random.uniform(0.8, 1.2)  # Random scaling factor between 0.8 and 1.2
        translation = np.random.uniform(-10, 10, size=2)  # Random translation vector between -10 and 10 in both directions
        rotation = np.random.uniform(-15, 15)  # Random rotation angle between -15 and 15 degrees
        shear = np.random.uniform(-0.2, 0.2, size=2)  # Random shear factor between -0.2 and 0.2 in both directions

        # Define the affine matrix
        affine_matrix = np.array([[scale * np.cos(rotation), -shear[0] * scale * np.sin(rotation), translation[0]],
                                  [shear[1] * scale * np.sin(rotation), scale * np.cos(rotation), translation[1]],
                                  [0, 0, 1]])
        x1_arr=affine_transform(x0_arr, affine_matrix)
        x1_arr=pca.fit_transform(x1_arr)
        x0 = torch.from_numpy(x0_arr).float()
        x1 = torch.from_numpy(x1_arr).float()
        x0 = x0.unsqueeze(0).to(device)
        x1 = x1.unsqueeze(0).to(device)

        # x1=torchvision.transforms.RandomAffine(0)(x0)
        x0_new = x0.view(feats.shape[0], 1, 128)
        x1_new = x1.view(feats.shape[0], 1, 128)
        z0, p0 = model_simsiam(x0_new, True)

        z1, p1 = model_simsiam(x1_new, True)
        loss = 0.5 * (criterion(z0, p1) + criterion(z1, p0))
        total_loss += loss.detach()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    avg_val_loss = total_loss / len(inputs)
    print(f"epoch: {epoch:>02}, loss: {avg_val_loss:.5f}")
    if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            torch.save(model_simsiam.state_dict(), model_path)
            print("Saved Best Model! in epoch", epoch+1)
    else:
        print("Weigh not updated in epoch", epoch+1)



Starting Training
epoch 1


  0%|          | 0/1000 [00:00<?, ?it/s]/home/phdcs2/Hard_Disk/Projects/T2I/deep-spectral-segmentation/venv/lib/python3.8/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
  0%|          | 3/1000 [00:03<13:24,  1.24it/s]/home/phdcs2/Hard_Disk/Projects/T2I/deep-spectral-segmentation/venv/lib/python3.8/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
  1%|          | 7/1000 [00:03<04:29,  3.69it/s]/home/phdcs2/Hard_Disk/Projects/T2I/deep-spectral-segmentation/venv/lib/python3.8/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
  1%|          | 9/1000 [00:04<03:27,  4.78it/s]/home/phdcs2/Hard_Disk/Projects/T2I/deep-spectral-segmentation/venv/lib/python3.8

epoch: 00, loss: -0.49695
Saved Best Model! in epoch 1
epoch 2


  0%|          | 3/1000 [00:00<02:47,  5.95it/s]/home/phdcs2/Hard_Disk/Projects/T2I/deep-spectral-segmentation/venv/lib/python3.8/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
  0%|          | 4/1000 [00:00<02:28,  6.72it/s]/home/phdcs2/Hard_Disk/Projects/T2I/deep-spectral-segmentation/venv/lib/python3.8/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
  1%|          | 8/1000 [00:01<02:37,  6.29it/s]/home/phdcs2/Hard_Disk/Projects/T2I/deep-spectral-segmentation/venv/lib/python3.8/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
  1%|          | 11/1000 [00:01<02:33,  6.43it/s]/home/phdcs2/Hard_Disk/Projects/T2I/deep-spectral-segmentation/venv/lib/

epoch: 01, loss: -0.57550
Saved Best Model! in epoch 2
epoch 3


...

epoch: 07, loss: -0.76028
Saved Best Model! in epoch 8
epoch 9


  0%|          | 0/1000 [00:00<?, ?it/s]/home/phdcs2/Hard_Disk/Projects/T2I/deep-spectral-segmentation/venv/lib/python3.8/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/home/phdcs2/Hard_Disk/Projects/T2I/deep-spectral-segmentation/venv/lib/python3.8/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
  0%|          | 4/1000 [00:00<02:53,  5.73it/s]/home/phdcs2/Hard_Disk/Projects/T2I/deep-spectral-segmentation/venv/lib/python3.8/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
  1%|          | 7/1000 [00:01<02:58,  5.57it/s]/home/phdcs2/Hard_Disk/Projects/T2I/deep-spectral-segmentation/venv/lib/python3.8/site-packages/sklearn/decomposition/_pca.py:545

epoch: 08, loss: -0.78807
Saved Best Model! in epoch 9
epoch 10


  0%|          | 4/1000 [00:00<03:17,  5.04it/s]/home/phdcs2/Hard_Disk/Projects/T2I/deep-spectral-segmentation/venv/lib/python3.8/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
  0%|          | 5/1000 [00:01<03:27,  4.79it/s]/home/phdcs2/Hard_Disk/Projects/T2I/deep-spectral-segmentation/venv/lib/python3.8/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
  1%|          | 7/1000 [00:01<03:05,  5.36it/s]/home/phdcs2/Hard_Disk/Projects/T2I/deep-spectral-segmentation/venv/lib/python3.8/site-packages/sklearn/decomposition/_pca.py:545: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
  2%|▏         | 15/1000 [00:02<02:29,  6.57it/s]/home/phdcs2/Hard_Disk/Projects/T2I/deep-spectral-segmentation/venv/lib/

epoch: 09, loss: -0.81873
Saved Best Model! in epoch 10


## Loading Model

In [ ]:
# torch.save(model_simsiam.state_dict(), model_path)
# print("Saved Best Model!")

# model_simsiam.load_state_dict(torch.load(model_path, map_location=CFG.device))

In [17]:
utils.make_output_dir(output_dir)
feat_list=[]
inputs = list(enumerate(sorted(Path(features_dir).iterdir())))
for inp in tqdm(inputs):
    index, features_file = inp
    # print(index, features_file)
     # Load
    data_dict = torch.load(features_file, map_location='cpu')
    # print(data_dict.keys())   #['k', 'indices', 'file', 'id', 'model_name', 'patch_size', 'shape']
    # print("shape=", data_dict['shape'], "k shape", data_dict['k'].shape, "patch_size=", data_dict['patch_size'])
    image_id = data_dict['file'][:-4]
    print(image_id)
    # Load
    output_file = str(Path(output_dir) / f'{image_id}.pth')
    if Path(output_file).is_file():
        print(f'Skipping existing file {str(output_file)}')
        # break
        # return  # skip because already generated

    # Load affinity matrix
    feats = data_dict[which_features].squeeze().cuda()
    # print("Without normalizing, Features Shape is",feats.shape)
    if normalize:
        feats = F.normalize(feats, p=2, dim=-1)
    # print("After normalization, Features Shape",feats.shape)
    # print("which_matrix=", which_matrix)
    # Eigenvectors of affinity matrix
    if which_matrix == 'affinity_torch':
        W = feats @ feats.T
        # W_feat=contrastive_affinity(feats, feats.T)
        # print("W shape=", W.shape)
        if threshold_at_zero:
            W = (W * (W > 0))
            # print("W shape=", W.shape)
        eigenvalues, eigenvectors = torch.eig(W, eigenvectors=True)
        eigenvalues = eigenvalues.cpu()
        eigenvectors = eigenvectors.cpu()
        print("which matrix=",which_matrix, "eigenvalues shape", eigenvalues.shape, "eigenvectors shape", eigenvectors.shape)


    # Eigenvectors of affinity matrix with scipy
    elif which_matrix == 'affinity_svd':
        USV = torch.linalg.svd(feats, full_matrices=False)
        eigenvectors = USV[0][:, :K].T.to('cpu', non_blocking=True)
        eigenvalues = USV[1][:K].to('cpu', non_blocking=True)
        print("which matrix=",which_matrix,"eigenvalues shape", eigenvalues.shape, "eigenvectors shape", eigenvectors.shape)

    # Eigenvectors of affinity matrix with scipy
    elif which_matrix == 'affinity':
        # print("Without normalizing, Features Shape is",feats.shape)
        W = (feats @ feats.T)
        # W_feat=contrastive_affinity(feats, feats.T)
        # print("W shape=", W.shape)
        if threshold_at_zero:
            W = (W * (W > 0))
        W = W.cpu().numpy()
        # print("W shape=", W.shape)
        eigenvalues, eigenvectors = eigsh(W, which='LM', k=K)
        eigenvectors = torch.flip(torch.from_numpy(eigenvectors), dims=(-1,)).T
        print("which matrix=",which_matrix, "eigenvalues shape", eigenvalues.shape, "eigenvectors shape", eigenvectors.shape)

    # Eigenvectors of matting laplacian matrix
    elif which_matrix in ['matting_laplacian', 'laplacian']:

        # Get sizes
        B, C, H, W, P, H_patch, W_patch, H_pad, W_pad = utils.get_image_sizes(data_dict)
        if image_downsample_factor is None:
            image_downsample_factor = P
        H_pad_lr, W_pad_lr = H_pad // image_downsample_factor, W_pad // image_downsample_factor

        # Upscale features to match the resolution
        if (H_patch, W_patch) != (H_pad_lr, W_pad_lr):
            feats = F.interpolate(
                feats.T.reshape(1, -1, H_patch, W_patch),
                size=(H_pad_lr, W_pad_lr), mode='bilinear', align_corners=False
            ).reshape(-1, H_pad_lr * W_pad_lr).T

        ### Feature affinities
        # print("Without normalizing, Features Shape is",feats.shape)

        W_feat_ds = (feats @ feats.T)
        # feat_list.append(feats)
        # feat_dataset = Feature_Dataset(feats)
        # if feats.shape[0]%2==0:
        #     features_dataloader = DataLoader(feat_dataset, batch_size=batch_size, shuffle=True)
        # else:
        #     features_dataloader = DataLoader(feat_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
        # model_simsiam = SimSiam()
        # device = "cuda" if torch.cuda.is_available() else "cpu"
        # model_simsiam.to(device)
        # criterion = NegativeCosineSimilarity()
        # optimizer = torch.optim.SGD(model_simsiam.parameters(), lr=0.06)
        #
        # print("type(features_dataloader", type(features_dataloader))
        # print(features_dataloader.shape)
        # print("Starting Training")
        # for epoch in range(epochs):
        #     total_loss = 0
        #     for x0 in features_dataloader:
        #     # for (x0), _, _ in features_dataloader:
        #     #     print("Before Unsqueezing, x0 shape=", x0.shape)
        #         x0 = x0.unsqueeze(0).to(device)
        #         # x0 = x0.unsqueeze(1).to(device)
        #         # print("After Unsqueezing x0 shape=", x0.shape)
        #         x1=torchvision.transforms.RandomAffine(0)(x0)
        #         # print("After Unsqueezing x1 shape=", x1.shape)
        #         # x0 = x0.squeeze(0).to(device)
        #         # print("batch_size=",batch_size)
        #         # x0_new = torch.tensor(x0).view(batch_size, 1, 1, 384)
        #         # x0_new = torch.tensor(x0).view(batch_size, 1, 384)
        #         x0_new = x0.view(batch_size, 1, 384)
        #         # print("After viewing x0 shape=", x0_new.shape)
        #         # print("x0.shape=", x0.shape)
        #         # x1 = x1.squeeze(0).to(device)
        #         # x1 = torch.tensor(x1).view(batch_size, 1, 1,384)
        #         x1_new = x1.view(batch_size, 1, 384)
        #         # print("After viewing x1 shape=", x1_new.shape)
        #         z0, p0 = model_simsiam(x0_new, True)
        #         # print("zo.shape", z0.shape)
        #         # print("p0.shape", p0.shape)
        #         z1, p1 = model_simsiam(x1_new, True)
        #         loss = 0.5 * (criterion(z0, p1) + criterion(z1, p0))
        #         total_loss += loss.detach()
        #         loss.backward()
        #         optimizer.step()
        #         optimizer.zero_grad()
        #     avg_loss = total_loss / len(features_dataloader)
        #     print(f"epoch: {epoch:>02}, loss: {avg_loss:.5f}")
        x0=feats
        x0_arr=x0.cpu()
        x0_arr= pca.fit_transform(x0_arr)
        x0 = torch.from_numpy(x0_arr).float()
        x0=x0.unsqueeze(1).to(device)
        # feats=feats.unsqueeze(1).to(device)
        # feats=feats.unsqueeze(2).to(device)
        # print("After Unsqueezing, feature size=", feats.shape)
        proj_pred_feature=model_simsiam(x0, False)
        projected_feature=proj_pred_feature[0]
        projected_feature = projected_feature.squeeze().to(device)
        # print("type(projected_feature)", type(projected_feature))
        print("projected feature shape=", projected_feature.shape)
        W_feat_siam=torch.matmul(projected_feature, projected_feature.t())
        W_feat=W_feat_ds + 0.1*W_feat_siam
        # print("W_feat.shape=", W_feat.shape)
        # print("W_feat.shape=", W_feat.shape)
        # W_feat=contrastive_affinity(feats, feats.T)
        if threshold_at_zero:
            W_feat = (W_feat * (W_feat > 0))
        W_feat = W_feat / W_feat.max()  # NOTE: If features are normalized, this naturally does nothing
        # W_feat = W_feat.cpu().numpy()
        W_feat = W_feat.detach().cpu().numpy()
        # print("W_feat shape=",W_feat.shape)

        ### Color affinities
        # If we are fusing with color affinites, then load the image and compute
        if image_color_lambda > 0:

            # Load image
            image_file = str(Path(images_root) / f'{image_id}.jpg')
            image_lr = Image.open(image_file).resize((W_pad_lr, H_pad_lr), Image.BILINEAR)
            image_lr = np.array(image_lr) / 255.

            # Color affinities (of type scipy.sparse.csr_matrix)
            if which_color_matrix == 'knn':
                W_lr = utils.knn_affinity(image_lr / 255)
            elif which_color_matrix == 'rw':
                W_lr = utils.rw_affinity(image_lr / 255)

            # Convert to dense numpy array
            W_color = np.array(W_lr.todense().astype(np.float32))
            # print("W_color shape", W_color.shape)

        else:

            # No color affinity
            W_color = 0

        # Combine
        W_comb = W_feat + W_color * image_color_lambda  # combination
        D_comb = np.array(utils.get_diagonal(W_comb).todense())  # is dense or sparse faster? not sure, should check
        # print("W_comb shape= ", W_comb.shape, "D_comb shape",  D_comb.shape)
        if lapnorm:
            try:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=K, sigma=0, which='LM', M=D_comb)
            except:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=K, which='SM', M=D_comb)
        else:
            try:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=K, sigma=0, which='LM')
            except:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=K, which='SM')
        eigenvalues, eigenvectors = torch.from_numpy(eigenvalues), torch.from_numpy(eigenvectors.T).float()
    print("eigenvalues shape", eigenvalues.shape, "eigenvectors shape", eigenvectors.shape)
    # Sign ambiguity
    for k in range(eigenvectors.shape[0]):
        if 0.5 < torch.mean((eigenvectors[k] > 0).float()).item() < 1.0:  # reverse segment
            eigenvectors[k] = 0 - eigenvectors[k]

    # Save dict
    output_dict = {'eigenvalues': eigenvalues, 'eigenvectors': eigenvectors}
    torch.save(output_dict, output_file)

  0%|          | 0/1000 [00:00<?, ?it/s]

0001
projected feature shape= torch.Size([400, 128])


  0%|          | 1/1000 [00:01<17:10,  1.03s/it]

eigenvalues shape torch.Size([5]) eigenvectors shape torch.Size([5, 400])
0002
projected feature shape= torch.Size([400, 128])


  0%|          | 2/1000 [00:01<10:06,  1.64it/s]

...

eigenvalues shape torch.Size([5]) eigenvectors shape torch.Size([5, 450])
0999
projected feature shape= torch.Size([375, 128])


100%|█████████▉| 999/1000 [10:25<00:01,  1.01s/it]

eigenvalues shape torch.Size([5]) eigenvectors shape torch.Size([5, 375])
1000
projected feature shape= torch.Size([400, 128])


100%|██████████| 1000/1000 [10:25<00:00,  1.60it/s]

eigenvalues shape torch.Size([5]) eigenvectors shape torch.Size([5, 400])


In [18]:
torch.cuda.empty_cache()